## .pth to .onnx

In [1]:
import torch
from torchvision import models

#### Let Op!
Deze stap is alleen nodig op het moment dat je een voorgetraind `.pth` (of `.pt`) bestand hebt die je wilt omzetten naar `.onnx` om in mendix te gebruiken.

In [3]:
def convert_pth_to_onnx(model_pth_path, onnx_output_path, dummy_input, model, num_classes):
    """
    Converteert een PyTorch resnet50 `state_dict` in een `.pth` bestand naar een ONNX modelbestand.

    :param model_pth_path: Pad naar de PyTorch `.pth` bestand (state_dict).
    :param onnx_output_path: Pad waar het ONNX-bestand moet worden opgeslagen.
    :param dummy_input: Een voorbeeldinvoer die de verwachte invoer voor het model simuleert.
                        Dit moet een torch.Tensor zijn met dezelfde vorm als de modelinvoer.
    """
    # Controleer of CUDA beschikbaar is
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Pas de volledig verbonden laag aan om te voldoen aan het aantal klassen
    model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
    model = model.to(device)
    
    # Laad het state_dict in het model
    state_dict = torch.load(model_pth_path, map_location=device)
    model.load_state_dict(state_dict)
    model.eval()  # Zet model in evaluatiemodus

    # Zorg ervoor dat de dummy input ook naar hetzelfde apparaat verplaatst
    dummy_input = dummy_input.to(device)

    # Exporteer naar ONNX
    torch.onnx.export(
        model,                # Het getrainde model
        dummy_input,          # Een dummy-invoer voor het model
        onnx_output_path,     # Waar het ONNX-bestand moet worden opgeslagen
        export_params=True,   # Exporteer ook de parameters van het model
        opset_version=11,     # De versie van het ONNX-opset om te gebruiken
        do_constant_folding=True,  # Optimaliseer constant-folding
        input_names=['input'],    # Naam van de input in ONNX-grafiek
        output_names=['output'],  # Naam van de output in ONNX-grafiek
        dynamic_axes={'input': {0: 'batch_size'},    # Dynamische batchgrootte
                      'output': {0: 'batch_size'}}
    )

    print(f"Model succesvol geconverteerd en opgeslagen als {onnx_output_path}.")

#### Vul onderstaande informatie in

In [5]:
input_size = 224
num_classes = 6
model_pth_path = 'path\\naar\\jou\\model.pth'
onnx_output_path = 'path\\naar\\jou\\model.onnx'

In [ ]:
dummy_input = torch.randn(1, 3, input_size, input_size)  # Afhankelijk van het model
model = models.resnet50()

convert_pth_to_onnx(model_pth_path, onnx_output_path, dummy_input, model, num_classes)